In [1]:
import sqlite3
from sqlite3 import Error
import os

path = os.getcwd()


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)


def create_project(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO projects(name,begin_date,end_date)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid


def create_task(conn, task):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = ''' INSERT INTO tasks(name,priority,status_id,project_id,begin_date,end_date)
              VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid


In [2]:
sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    begin_date text,
                                    end_date text
                                ); """

sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                id integer PRIMARY KEY,
                                name text NOT NULL,
                                priority integer,
                                status_id integer NOT NULL,
                                project_id integer NOT NULL,
                                begin_date text NOT NULL,
                                end_date text NOT NULL,
                                FOREIGN KEY (project_id) REFERENCES projects (id)
                            );"""

In [3]:
# create a database connection
conn = create_connection(path + "/pythonsqlite.db")

# create tables
if conn is not None:
    # create projects table
    create_table(conn, sql_create_projects_table)

    # create tasks table
    create_table(conn, sql_create_tasks_table)

    # create a new project
    project = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
    project_id = create_project(conn, project)

    # tasks
    task_1 = ('Analyze the requirements of the app', 1, 1, project_id, '2015-01-01', '2015-01-02')
    task_2 = ('Confirm with user about the top requirements', 1, 1, project_id, '2015-01-03', '2015-01-05')
    task_3 = ('Confirm with user about the top requirements', 2, 2, project_id, '2015-01-03', '2015-01-05')

    # create tasks
    create_task(conn, task_1)
    create_task(conn, task_2)
    create_task(conn, task_3)

else:
    print("Error! cannot create the database connection.")

In [4]:
cur = conn.cursor()
cur.execute("SELECT * FROM projects").fetchall()

[(1, 'Cool App with SQLite & Python', '2015-01-01', '2015-01-30'),
 (2, 'Cool App with SQLite & Python', '2015-01-01', '2015-01-30')]

In [5]:
cur.execute("SELECT * FROM tasks").fetchone()

(2,
 3,
 'Confirm with user about the top requirements',
 2,
 2,
 1,
 '2015-01-03',
 '2015-01-05')

In [6]:
cur.close()

In [7]:
import pandas as pd

# Read sqlite query results into a pandas DataFrame
tasks_df = pd.read_sql_query("SELECT * from tasks", conn)

# Verify that result of SQL query is stored in the dataframe
tasks_df.head()


,index,id,name,priority,status_id,project_id,begin_date,end_date
0,2.0,3.0,Confirm with user about the top requirements,2,2,1,2015-01-03,2015-01-05
1,NaN,NaN,Analyze the requirements of the app,1,1,2,2015-01-01,2015-01-02
2,NaN,NaN,Confirm with user about the top requirements,1,1,2,2015-01-03,2015-01-05
3,NaN,NaN,Confirm with user about the top requirements,2,2,2,2015-01-03,2015-01-05


In [8]:
# Select only data for status_id = 2
tasks_df = tasks_df[tasks_df.status_id == 2]

# Write the new DataFrame to a new SQLite table
tasks_df.to_sql("tasks", conn, if_exists="replace")


2

In [9]:
new_tasks_df = pd.read_sql_query("SELECT * from tasks", conn)
new_tasks_df

,level_0,index,id,name,priority,status_id,project_id,begin_date,end_date
0,0,2.0,3.0,Confirm with user about the top requirements,2,2,1,2015-01-03,2015-01-05
1,3,NaN,NaN,Confirm with user about the top requirements,2,2,2,2015-01-03,2015-01-05


In [10]:
# !pip3 install faker
from faker import Faker
import pandas as pd
fake = Faker()
profileData = [fake.simple_profile() for i in range(100)]
profiles = pd.DataFrame(profileData)

profiles.head()

,username,name,sex,address,mail,birthdate
0,ericjohnson,Jack Roberts,M,"PSC 7674, Box 1931\nAPO AA 44469",fsandoval@yahoo.com,1930-09-17
1,andrew91,James Velasquez,M,"940 Robertson Rue Apt. 779\nWashingtonport, ID...",brownvalerie@gmail.com,1993-02-01
2,landrygerald,Lisa Edwards,F,"1476 Burke Falls Apt. 512\nNovakfurt, TN 78484",jasongardner@hotmail.com,1985-07-15
3,christophergonzales,Tony Dunlap,M,"07341 Stephanie Turnpike\nSouth Kristinbury, W...",markedwards@hotmail.com,1910-02-26
4,zacharycohen,Mary Duncan,F,"538 Sharon Wall Suite 419\nNorth Gregory, PW 6...",paul40@yahoo.com,1927-11-01


In [11]:
profiles.columns.tolist()

['username', 'name', 'sex', 'address', 'mail', 'birthdate']

In [12]:
conn.execute('CREATE TABLE IF NOT EXISTS profiles (username, name, sex, address, mail, birthdate)')
conn.commit()
profiles.to_sql('profiles', conn, if_exists='replace', index = False)

profiles_df = pd.read_sql_query("SELECT * from profiles", conn)
profiles_df.head()

,username,name,sex,address,mail,birthdate
0,ericjohnson,Jack Roberts,M,"PSC 7674, Box 1931\nAPO AA 44469",fsandoval@yahoo.com,1930-09-17
1,andrew91,James Velasquez,M,"940 Robertson Rue Apt. 779\nWashingtonport, ID...",brownvalerie@gmail.com,1993-02-01
2,landrygerald,Lisa Edwards,F,"1476 Burke Falls Apt. 512\nNovakfurt, TN 78484",jasongardner@hotmail.com,1985-07-15
3,christophergonzales,Tony Dunlap,M,"07341 Stephanie Turnpike\nSouth Kristinbury, W...",markedwards@hotmail.com,1910-02-26
4,zacharycohen,Mary Duncan,F,"538 Sharon Wall Suite 419\nNorth Gregory, PW 6...",paul40@yahoo.com,1927-11-01


In [13]:
import random

salaries = pd.DataFrame({"username":profiles.username, 
                         "salary": random.sample(range(100000,500000),len(profiles.username))})

salaries.head()

,username,salary
0,ericjohnson,102481
1,andrew91,223055
2,landrygerald,482865
3,christophergonzales,234330
4,zacharycohen,273210


In [14]:
salaries.columns.tolist()

['username', 'salary']

In [15]:
conn.execute('CREATE TABLE IF NOT EXISTS salaries (username, salary)')
conn.commit()
salaries.to_sql('salaries', conn, if_exists='replace', index = False)

salaries_df = pd.read_sql_query("SELECT * from salaries", conn)
salaries_df.head()

,username,salary
0,ericjohnson,102481
1,andrew91,223055
2,landrygerald,482865
3,christophergonzales,234330
4,zacharycohen,273210


In [16]:
inner_join = pd.read_sql_query("SELECT * from salaries s inner join profiles p on p.username == s.username", conn)
inner_join

,username,salary,username,name,sex,address,mail,birthdate
0,ericjohnson,102481,ericjohnson,Jack Roberts,M,"PSC 7674, Box 1931\nAPO AA 44469",fsandoval@yahoo.com,1930-09-17
1,andrew91,223055,andrew91,James Velasquez,M,"940 Robertson Rue Apt. 779\nWashingtonport, ID...",brownvalerie@gmail.com,1993-02-01
2,landrygerald,482865,landrygerald,Lisa Edwards,F,"1476 Burke Falls Apt. 512\nNovakfurt, TN 78484",jasongardner@hotmail.com,1985-07-15
3,christophergonzales,234330,christophergonzales,Tony Dunlap,M,"07341 Stephanie Turnpike\nSouth Kristinbury, W...",markedwards@hotmail.com,1910-02-26
4,zacharycohen,273210,zacharycohen,Mary Duncan,F,"538 Sharon Wall Suite 419\nNorth Gregory, PW 6...",paul40@yahoo.com,1927-11-01
...,...,...,...,...,...,...,...,...
95,emily35,489924,emily35,Juan Brown,M,USNV Hines\nFPO AE 46191,richardsonemily@yahoo.com,1992-05-27
96,xcooper,370332,xcooper,William Davis,M,"10861 Susan Creek Suite 556\nPort Markmouth, L...",pshaw@gmail.com,1918-10-08
97,dustinbecker,405974,dustinbecker,Edward Mitchell,M,"696 Hess Hollow Apt. 475\nAndersonstad, VT 33864",lewisthomas@hotmail.com,1968-07-15
98,kedwards,192144,kedwards,Casey Coleman,M,"485 Conley Street Suite 024\nEast Grant, IA 75818",alexander31@gmail.com,1974-04-13
